In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-31"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15330,2024-03-31,Espanha Acb,08:00,Gran Canaria,Valencia,1.50,2.61,163.5,1.81,1.97,-4.5,1.89,3.20,6qpzlJfC,0.666667,0.383142,0.552486,0.507614,0.049808,0.6,0.6,NaN,NaN,151.442,75.954822,0.501544,161.820,79.782763,0.493034,143.232,241.104,281.00,128.70,0.0,0.0,0.0,0.0,0.381941,0.059861,0.363972,2.65,0.530,0.943396,0.000000,0.0,0.000000,3.02,0.604,2.665563,0.000000,0.0,0.000000
15331,2024-03-31,Espanha Acb,12:00,Granada,Joventut,2.12,1.73,164.5,1.83,1.95,1.5,1.97,1.58,2BwqncPO,0.471698,0.578035,0.546448,0.512821,0.049733,0.6,0.4,NaN,NaN,212.980,106.223386,0.498748,185.704,75.963452,0.409057,160.068,206.784,140.07,127.09,0.0,0.0,0.0,0.0,0.143258,0.044896,0.155364,-0.41,-0.082,-13.658537,0.000000,0.0,0.000000,0.85,0.170,4.294118,0.000000,0.0,0.000000
15332,2024-03-31,Espanha Acb,13:30,Obradoiro CAB,Barcelona,4.23,1.23,165.5,1.83,1.93,9.5,1.84,1.04,GERQAt2P,0.236407,0.813008,0.546448,0.518135,0.049415,0.6,0.4,NaN,NaN,232.178,76.990824,0.331603,122.362,26.675142,0.218002,234.426,132.968,248.82,166.08,0.0,0.0,0.0,0.0,0.777040,0.037612,0.392837,-5.00,-1.000,-3.230000,0.000000,0.0,0.000000,-0.71,-0.142,-1.619718,0.000000,0.0,0.000000
15333,2024-03-31,Eua Nba,20:00,Houston Rockets,Dallas Mavericks,2.05,1.82,229.5,1.83,1.93,1.5,1.95,1.67,r7YfxixE,0.487805,0.549451,0.546448,0.518135,0.037255,0.4,0.4,NaN,NaN,196.750,44.354430,0.225435,218.716,43.934788,0.200876,242.752,236.606,217.17,206.51,0.0,0.0,0.0,0.0,0.084049,0.037612,0.109387,3.09,0.618,1.699029,0.493488,0.6,0.106512,2.30,0.460,1.782609,0.537329,0.7,0.162671
15334,2024-03-31,Eua Nba,20:00,New York Knicks,Oklahoma City Thunder,1.75,2.15,215.5,1.83,1.93,-3.5,2.03,2.54,6kkCD7aD,0.571429,0.465116,0.546448,0.518135,0.036545,0.2,0.2,NaN,NaN,212.978,31.819710,0.149404,187.644,32.673409,0.174124,179.762,195.276,170.52,197.54,0.0,0.0,0.0,0.0,0.145048,0.037612,0.157823,-2.18,-0.436,-1.720183,0.643713,0.7,0.056287,-0.10,-0.020,-57.500000,0.613611,0.5,-0.113611
15335,2024-03-31,Alemanha Bbl,12:00,Ludwigsburg,Ulm,1.63,2.29,174.5,1.83,1.95,-3.5,1.95,2.54,MPRQZlx5,0.613497,0.436681,0.546448,0.512821,0.050178,0.4,0.6,NaN,NaN,141.438,28.411044,0.200873,176.886,88.149074,0.498338,120.426,190.060,129.22,179.82,0.0,0.0,0.0,0.0,0.238107,0.044896,0.185832,0.01,0.002,315.000000,0.752719,0.7,-0.052719,-2.27,-0.454,-2.841410,0.556600,0.6,0.043400
15336,2024-03-31,Alemanha Bbl,13:30,Rostock,Oldenburg,2.17,1.70,175.5,1.81,1.95,1.5,2.00,1.54,8tQYXS6H,0.460829,0.588235,0.552486,0.512821,0.049065,0.6,0.4,NaN,NaN,205.176,37.343916,0.182009,232.234,149.158611,0.642277,211.944,287.844,158.34,269.10,0.0,0.0,0.0,0.0,0.171752,0.052657,0.183768,-5.00,-1.000,-1.170000,0.558974,0.4,-0.158974,-2.61,-0.522,-1.340996,0.000000,0.0,0.000000
15337,2024-03-31,Ásia Copa Asiática Feminina 3X3,01:45,Japão 3x3 F,Nova Zelândia 3x3 F,2.72,1.38,32.5,1.83,1.83,-1.5,1.83,1.85,A9lcBLNu,0.367647,0.724638,0.546448,0.546448,0.092285,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.462206,0.000000,0.007686,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
15338,2024-03-31,Bulgária Nbl,07:00,Balkan,Rilski Sportist,2.12,1.67,159.5,1.81,1.85,1.5,2.00,1.51,t8yULPJj,0.471698,0.598802,0.552486,0.540541,0.070501,0.8,0.8,NaN,NaN,109.832,19.689481,0.179269,108.442,16.851487,0.155396,124.594,128.462,137.74,100.05,0.0,0.0,0.0,0.0,0.167915,0.015456,0.197426,0.61,0.122,9.180328,0.000000,0.0,0.000000,2.17,0.434,1.543779,0.000000,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,08:00,Espanha Acb,Gran Canaria,Valencia,163.5,1.81,1.0000,Over
1,20:00,Eua Nba,New York Knicks,Oklahoma City Thunder,215.5,1.83,1.0000,Over
2,12:00,Alemanha Bbl,Ludwigsburg,Ulm,174.5,1.83,1.0000,Over
3,08:35,China Cba,Zhejiang Guangsha,Qingdao,199.5,1.83,1.0000,Over
4,06:00,Coreia Do Sul Kbl,Goyang Sono,Suwon KT,171.5,1.80,1.0000,Over
5,06:00,Coreia Do Sul Kbl,KCC Egis,Seoul Knights,168.5,1.84,1.0000,Over
6,06:00,Coreia Do Sul Kbl,Mobis Phoebus,Anyang JungKwanJang,177.5,1.88,1.0000,Over
7,12:00,Eslováquia Extraliga,Inter Bratislava,BC Komarno,162.5,1.85,0.9997,Over
8,07:00,Espanha Liga Feminina,Ensino F,Ferrol F,135.5,1.85,1.0000,Over
9,08:00,Espanha Liga Feminina,Gran Canaria F,Jairis F,136.5,1.85,0.9999,Over
